In [1]:
import pandas as pd
import numpy as np
import os
import re

# Processing utilities

In [2]:
B_PYTHON, E_PYTHON = '[PYTHON]', '[/PYTHON]'
B_CPP, E_CPP = '[CPP]', '[/CPP]'
B_INST, E_INST = '[INST]', '[/INST]'

In [3]:
cpp_func_match_reg_exp = "\\b[A-Za-z_][A-Za-z0-9_]*\s+[A-Za-z_][A-Za-z0-9_]*\s*\([^;{]*\)\s*{"

In [4]:
def end_overlap(a, b):
    for i in range(0, len(a)):
        if b.startswith(a[i:]):
            return i
    return len(a)

def remove_prefix(s, prefix):
    prefix_end_idx = s.index(prefix) + len(prefix)
    return s[prefix_end_idx:]

def clean_code(code):
    # cleans lines like these where appending of code context and response creates - def CalculateDifference(arr, n):def calculate_difference(arr, n):
    lines = code.split('\n')
    cleaned_code = []
    for line in lines:
        if line.count('def') == 2:
            code_context, response = line.split(':', 1)
            response = response.strip()
            cleaned_code.append(response)
        else:
            cleaned_code.append(line)

    return '\n'.join(cleaned_code)

def clean_duplicate_func_signature(code):
    """
    for cpp files - clean lines like the following which have duplicate function signatures - 

        int countSol ( int coeff [ ] , int start , int end , int rhs ) {
        int countSol(int coeff[], int start, int end, int rhs) {
    
    """
    lines = code.split('\n')
    cleaned_code = []
    for i, line in enumerate(lines):
        match = re.search(cpp_func_match_reg_exp, line)
        if match and i < len(lines)-1:
            second_match = re.search(cpp_func_match_reg_exp, lines[i+1])
            if not second_match:
                cleaned_code.append(line)
        else:
            cleaned_code.append(line)

    return '\n'.join(cleaned_code)

def extract_code_python(prompt, response, code_context):
    trimmed_response = remove_prefix(response, prompt)
    if trimmed_response.startswith(B_PYTHON):
        trimmed_response = remove_prefix(trimmed_response, B_PYTHON)
    try:
        suffix_idx = trimmed_response.index(E_PYTHON)
        trimmed_response = trimmed_response[:suffix_idx]
    except ValueError as ve:
          pass
    trimmed_response = trimmed_response.strip()
    code_context = code_context.strip()
    overlap_idx = end_overlap(code_context, trimmed_response)
    cleaned_response = code_context[:overlap_idx] + trimmed_response
    return clean_code(cleaned_response)

def extract_code_cpp(prompt, response, code_context):
    trimmed_response = remove_prefix(response, prompt)
    if trimmed_response.startswith(B_CPP):
        trimmed_response = remove_prefix(trimmed_response, B_CPP)
    try:
        suffix_idx = trimmed_response.index(E_CPP)
        trimmed_response = trimmed_response[:suffix_idx]
    except ValueError as ve:
          pass

    try:
        suffix_idx = trimmed_response.index(E_INST)
        trimmed_response = trimmed_response[:suffix_idx]
    except ValueError as ve:
          pass

    trimmed_response = trimmed_response.strip()
    code_context = code_context.strip()
    overlap_idx = end_overlap(code_context, trimmed_response)
    cleaned_response = code_context[:overlap_idx] + '\n' + trimmed_response
    return clean_duplicate_func_signature(clean_code(cleaned_response))

def save_file(path, code):
    with open(path, 'w', encoding='utf-8') as file:
        file.write(code)

def save_to_code_file(directory, id, code, ext, pss=1):
    save_file(f"{directory}/question-{id}-pass-{pss}.{ext}", code)

# Python processing

In [6]:
python_responses_df = pd.read_csv('./python/train_response.csv')
python_responses_df.head()

,id,text,code,question,functions,code_context,context,llama_prompt,response
0,0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ...",Maximum Prefix Sum possible by merging two giv...,"['def maxPresum(a, b):']","def maxPresum(a, b):","def maxPresum(a, b):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
1,1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...,Check if a number can be represented as sum of...,['def sumOfTwoCubes(n):'],import math\n\n\ndef sumOfTwoCubes(n):,import math\n\n\ndef sumOfTwoCubes(n):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
2,2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,Generate an N.,"['def sieveOfPrimes():', 'def getArray(arr, N):']",sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
3,3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...,Nth natural number after removing all numbers ...,['def findNthNumber(N):'],def findNthNumber(N):,def findNthNumber(N):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
4,4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =...",Check if an integer is rotation of another giv...,"['def check(A, B):']","import math\n\n\ndef check(A, B):","import math\n\n\ndef check(A, B):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...


In [7]:
print(python_responses_df.loc[2]['response'])

<s>[INST]<<SYS>>
Provide answers in Python. Wrap around code in [PYTHON] and [/PYTHON]. Don't write any comments in generated code.
<</SYS>>

# Generate an N.
sieve = [1] * (1000000 + 1)


def sieveOfPrimes():[/INST]
[PYTHON]
def sieveOfPrimes():
    sieve = [1] * (1000000 + 1)
    sieve[0] = sieve[1] = 0
    for i in range(2, int(sieve ** 0.5) + 1):
        if sieve[i]:
            for j in range(i * i, len(sieve), i):
                sieve[j] = 0
    primes = []
    for i, is_prime in enumerate(sieve):
        if is_prime:
            primes.append(i)
    return primes[1000000]
[/PYTHON]

# Generate an N.
sieve = [1] * (1000000 + 1)


def sieveOfPrimes():[/INST]
[PYTHON]
def sieveOfPrimes():
    sieve = [1] * (1000000 + 1)
    sieve[0] = sieve[1] = 0
    for i in range(2, int(sieve ** 0.5) + 1):
        if sieve[i]:
            for j in range(i * i, len(sieve), i):
                sieve[j] = 0
    primes = []
    for i, is_prime in enumerate(sieve):
        if is_prime:
            p

In [145]:
def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [146]:
PYTHON_SAVE_PATH = './python/files'
create_directory_if_not_exists(PYTHON_SAVE_PATH)

In [147]:
python_responses_df['response_code'] = python_responses_df.apply(lambda row: extract_code_python(row.llama_prompt, row.response, row.code_context), axis=1)
python_responses_df.head()

,id,text,code,question,functions,code_context,context,llama_prompt,response,response_code
0,0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ...",Maximum Prefix Sum possible by merging two giv...,"['def maxPresum(a, b):']","def maxPresum(a, b):","def maxPresum(a, b):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,"def maxPresum(a, b):\n max_sum = 0\n for..."
1,1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...,Check if a number can be represented as sum of...,['def sumOfTwoCubes(n):'],import math\n\n\ndef sumOfTwoCubes(n):,import math\n\n\ndef sumOfTwoCubes(n):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,import math\n\n\ndef sumOfTwoCubes(n):\n fo...
2,2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,Generate an N.,"['def sieveOfPrimes():', 'def getArray(arr, N):']",sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...
3,3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...,Nth natural number after removing all numbers ...,['def findNthNumber(N):'],def findNthNumber(N):,def findNthNumber(N):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,def findNthNumber(N):\n nth_number = 1\n ...
4,4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =...",Check if an integer is rotation of another giv...,"['def check(A, B):']","import math\n\n\ndef check(A, B):","import math\n\n\ndef check(A, B):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,"import math\n\n\ndef check(A, B):\n if A ==..."


In [148]:
python_responses_df.to_csv('./python/train_response_processed.csv')

In [149]:
python_responses_df.apply(lambda row: save_to_code_file(PYTHON_SAVE_PATH, row.id, row.response_code, "py"), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
295    None
296    None
297    None
298    None
299    None
Length: 300, dtype: object

In [37]:
# random_samples = python_responses_df['response_code'].sample(20).values.tolist()

# for i, code in enumerate(random_samples):
#     print(i)
#     print(code, '\n')

In [38]:
# # def calculate_difference(arr, n)
# python_responses_df[python_responses_df['response_code'].str.contains('from math import sqrt')]

In [39]:
# print(python_responses_df.loc[288]['response'])

# C++ processing

In [131]:
cpp_responses_df = pd.read_csv('./cpp/train_response.csv')
cpp_responses_df.head()

,id,text,code,question,code_cleaned,functions,code_context,context,llama_prompt,response
0,0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
1,1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
2,3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
3,4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...
4,5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...


In [132]:
len(cpp_responses_df)

300

In [136]:
CPP_SAVE_PATH = './cpp/files'
create_directory_if_not_exists(CPP_SAVE_PATH)

In [133]:
cpp_responses_df['response_code'] = cpp_responses_df.apply(lambda row: extract_code_cpp(row.llama_prompt, row.response, row.code_context), axis=1)
cpp_responses_df.head()

,id,text,code,question,code_cleaned,functions,code_context,context,llama_prompt,response,response_code
0,0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...
1,1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...
2,3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...
3,4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...
4,5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,<s>[INST]<<SYS>>\nProvide answers in CPP. Wrap...,#include <bits/stdc++.h> \n using namespace st...


In [134]:
# random_samples = cpp_responses_df['response_code'].sample(20).values.tolist()

# for i, code in enumerate(random_samples):
#     print(i)
#     print(code, '\n')

In [135]:
cpp_responses_df.to_csv('./cpp/train_response_processed.csv')

In [150]:
cpp_responses_df.apply(lambda row: save_to_code_file(CPP_SAVE_PATH, row.id, row.response_code, "cpp"), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
295    None
296    None
297    None
298    None
299    None
Length: 300, dtype: object